# UI notebook

## This notebook runs the UI of the project 

--------------

In [0]:
# ============================================================
# FINAL UI: Linear Scoring Only (Hardcoded)
# ============================================================

from pyspark.sql import functions as F
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import base64
import random
from io import BytesIO

# ----------------------------
# 0) Load & Pre-Calculate Data
# ----------------------------
master_df = spark.read.format("delta").load("dbfs:/FileStore/osm_pipeline/final_listings_scored").cache()

city_stats_rows = master_df.groupBy("city").agg(
    F.min("price").alias("min_price"),
    F.max("price").alias("max_price"),
    F.percentile_approx("price", 0.95).alias("p95_price"),
    F.avg("price").alias("avg_price"),
    F.mean("Food_Drinks_score_norm").alias("avg_food"),
    F.mean("Outdoor_Attractions_score_norm").alias("avg_outdoor"),
    F.mean("Indoor_Culture_score_norm").alias("avg_culture"),
    F.mean("Daily_Essentials_score_norm").alias("avg_essentials"),
    F.mean("Shopping_score_norm").alias("avg_shopping")
).collect()

CITY_STATS = {row["city"]: row.asDict() for row in city_stats_rows}
ALL_CITIES = sorted(CITY_STATS.keys())

# ----------------------------
# 1) Config & Mappings
# ----------------------------
VIBE_CONFIG = {
    "Food_Drinks": { "col": "Food_Drinks_score_norm", "desc": "Food & Drinks" },
    "Outdoor_Attractions": { "col": "Outdoor_Attractions_score_norm", "desc": "Outdoor & Nature" },
    "Indoor_Culture": { "col": "Indoor_Culture_score_norm", "desc": "Indoor Culture" },
    "Daily_Essentials": { "col": "Daily_Essentials_score_norm", "desc": "Daily Essentials" },
    "Shopping": { "col": "Shopping_score_norm", "desc": "Shopping / Misc" }
}
VIBE_KEYS = list(VIBE_CONFIG.keys())

STYLE = {"description_width": "200px"}
LAYOUT = widgets.Layout(width="500px")

# ----------------------------
# 2) Helper Functions
# ----------------------------
def mk_slider(desc, v=5, mn=0, mx=10):
    return widgets.IntSlider(description=desc, min=mn, max=mx, step=1, value=v, style=STYLE, layout=LAYOUT)

def mk_dropdown(desc, options):
    return widgets.Dropdown(description=desc, options=options, style=STYLE, layout=LAYOUT)

def normalize_weights(raw_dict):
    total = sum(raw_dict.values())
    if total <= 0: return {k: 1.0/len(raw_dict) for k in raw_dict}
    return {k: v/total for k, v in raw_dict.items()}

def display_scrollable_df(df_pandas):
    """Render dataframe with horizontal scrollbar"""
    html = df_pandas.to_html(index=False, escape=False)
    display(HTML(f"""<div style="width:100%; overflow-x: auto; white-space: nowrap; margin-bottom: 20px;">{html}</div>"""))

def display_scrollable_plot(fig):
    """Render matplotlib figure as a scrollable HTML image"""
    buf = BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', dpi=100)
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode('utf-8')
    plt.close(fig)
    display(HTML(f"""
    <div style="width:100%; overflow-x: auto; white-space: nowrap; border: 1px solid #ddd; padding: 10px;">
        <img src="data:image/png;base64,{img_str}" style="max-width: none; height: 350px;">
    </div>
    """))

# ----------------------------
# 3) Widget Setup
# ----------------------------
city_dd = mk_dropdown("City:", ALL_CITIES)
budget_slider = widgets.IntSlider(description="Max Nightly Price ($):", min=0, max=1000, value=200, step=10, style=STYLE, layout=LAYOUT)

w_price = mk_slider("Price Sensitivity", v=7)
w_rating = mk_slider("Rating Importance", v=6)
w_amenities = mk_slider("Amenities Quantity", v=4)
w_vibe = mk_slider("Neighborhood Vibe", v=8)

v_sliders = {k: mk_slider(config["desc"], v=5) for k, config in VIBE_CONFIG.items()}

# REMOVED: func_dd (Formula Dropdown)

K_slider = widgets.IntSlider(description="Top Results (K):", min=1, max=10, value=5, step=1, style=STYLE, layout=LAYOUT)

explanation_panel = widgets.HTML("""
<div style="padding:15px; background-color:#2D2D2D; border-radius:10px; border:1px solid #444; font-size:13px; line-height:1.6; color:#ddd; height: 100%;">
  <h4 style="margin-top:0; color:#00A699;">💡 Weights Guide</h4>
  <b>Core weights</b> define your priorities (Cheap vs Quality vs Vibe).<br><br>
  <h4 style="color:#FF5A5F;">🏙 Vibe Definitions</h4>
  <b>🍽 Food & Drinks</b>: Cafés, restaurants, bars.<br>
  <b>🌳 Outdoor</b>: Parks, nature, views.<br>
  <b>🏛 Indoor Culture</b>: Museums, history.<br>
  <b>🛒 Essentials</b>: Grocery, pharmacy.<br>
  <b>🛍 Shopping</b>: Malls, retail.
</div>
""")

def on_city_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        city = change['new']
        stats = CITY_STATS.get(city, {"min_price": 0, "max_price": 1000})
        budget_slider.min = int(stats["min_price"])
        budget_slider.max = int(stats["max_price"]) 
        if budget_slider.value > int(stats["max_price"]): budget_slider.value = int(stats["max_price"])
city_dd.observe(on_city_change)
on_city_change({'type': 'change', 'name': 'value', 'new': city_dd.value})

# ----------------------------
# 4) Scoring Engine (Linear model - winner in the Evaluation stages)
# ----------------------------
def calculate_personal_score(df, params):
    core_w = normalize_weights({"wP": params["w_price"], "wQ": params["w_rating"], "wA": params["w_amenities"], "wV": params["w_vibe"]})
    vibe_w = normalize_weights(params["vibe_weights"])
    
    vibe_expr = F.lit(0.0)
    for cat, weight in vibe_w.items():
        col_name = VIBE_CONFIG[cat]["col"]
        if col_name in df.columns:
            vibe_expr = vibe_expr + (F.col(col_name) * weight)
    
    df = df.withColumn("V_user", vibe_expr)
    
    # Components
    if "WeightedRating_01" in df.columns: Q = F.col("WeightedRating_01")
    elif "ratings" in df.columns: Q = F.col("ratings") / 5.0
    else: Q = F.lit(0.5)

    if "amenityScore_01" in df.columns: A = F.col("amenityScore_01")
    elif "amenitiesCount" in df.columns: A = F.least(F.col("amenitiesCount") / 50.0, F.lit(1.0))
    else: A = F.lit(0.5)
    
    P_expensive = F.least(F.col("price") / params["budget"], F.lit(1.0))
    P_cheap = F.lit(1.0) - P_expensive 
    V = F.col("V_user")

    # --- HARDCODED LINEAR FORMULA ---
    # Score = wQ*Q + wA*A + wV*V + wP*(1-Price)
    # (Note: P_cheap is effectively (1-P), so this rewards lower prices)
    score_expr = (core_w["wQ"]*Q) + (core_w["wA"]*A) + (core_w["wV"]*V) + (core_w["wP"]*P_cheap)
        
    return df.withColumn("Personalized_Score", score_expr)

# ----------------------------
# 5) Run Logic & VISUALIZATION
# ----------------------------
out = widgets.Output()

def on_run_click(b):
    with out:
        clear_output()
        print("🔍 Searching...")
        
        params = {
            "city": city_dd.value, "budget": budget_slider.value,
            "w_price": w_price.value, "w_rating": w_rating.value,
            "w_amenities": w_amenities.value, "w_vibe": w_vibe.value,
            "vibe_weights": {k: v_sliders[k].value for k in VIBE_KEYS},
            # REMOVED: "formula": func_dd.value
            "k": K_slider.value
        }
        
        # Filter & Score
        df_filtered = master_df.filter((F.col("city") == params["city"]) & (F.col("price") <= params["budget"]))
        if df_filtered.count() == 0:
            print("❌ No listings found.")
            return

        df_scored = calculate_personal_score(df_filtered, params)
        df_top = df_scored.orderBy(F.desc("Personalized_Score")).limit(params["k"])
        
        # --- PREPARE USER TABLE ---
        col_map = {
            "property_id": "ID", "name": "Listing Name", "price": "Price ($)", "ratings": "Rating", 
            "guests": "Guests", "pets_allowed": "Pets Allowed", "amenitiesCount": "Amenities"
        }
        
        math_cols = ["Personalized_Score", "V_user"]
        for cat in VIBE_CONFIG.values():
            if cat["col"] in df_top.columns: math_cols.append(cat["col"])
            
        full_pdf = df_top.select(*(list(col_map.keys()) + math_cols)).toPandas()
        
        # --- Emojis for Pets ---
        if "pets_allowed" in full_pdf.columns:
            full_pdf["pets_allowed"] = full_pdf["pets_allowed"].apply(lambda x: "🐶" if x is True or str(x).lower() == 'true' else "❌")

        # Display Table
        display_pdf = full_pdf[list(col_map.keys())].rename(columns=col_map)
        print(f"\n🏆 Top {params['k']} Listings in {params['city']}")
        display_pdf["Price ($)"] = display_pdf["Price ($)"].apply(lambda x: f"{x:.2f}")
        display_scrollable_df(display_pdf)
        
        # --- PLOTTING ---
        if not full_pdf.empty:
            fig, ax = plt.subplots(1, 4, figsize=(24, 5))
            plt.subplots_adjust(wspace=0.5)
            
            # 1. Price vs Value
            full_pdf["ratings"] = pd.to_numeric(full_pdf["ratings"], errors='coerce').fillna(0)
            bubble_size = full_pdf["ratings"] * 40
            bubble_size = bubble_size.replace(0, 20)
            
            sc = ax[0].scatter(full_pdf["price"], full_pdf["Personalized_Score"], 
                             c=full_pdf["V_user"], s=bubble_size, 
                             cmap="coolwarm", alpha=0.8, edgecolors='k')
            
            # Annotations
            for i, row in full_pdf.iterrows():
                ax[0].text(row["price"], row["Personalized_Score"], str(i+1), 
                           fontsize=12, fontweight='bold', ha='center', va='bottom', color='black')

            # Dynamic Insight
            avg_top_p = full_pdf["price"].mean()
            corr = full_pdf["price"].corr(full_pdf["Personalized_Score"])
            insight_text = f"Top results avg ${int(avg_top_p)}"
            if corr < -0.3: insight_text += " (Cheaper is Better)"
            elif corr > 0.3: insight_text += " (Quality Costs More)"
            
            ax[0].set_title(f"Trade-off: Price vs Value\n({insight_text})")
            ax[0].set_xlabel("Price ($)")
            ax[0].set_ylabel("VFM Score")
            ax[0].grid(True, linestyle='--', alpha=0.5)
            cbar = plt.colorbar(sc, ax=ax[0])
            cbar.set_label("Vibe Match (0-1)")
            
            # 2. Winner Vibe Profile
            winner = full_pdf.iloc[0]
            vibe_vals = [winner[VIBE_CONFIG[k]["col"]] for k in VIBE_KEYS if VIBE_CONFIG[k]["col"] in full_pdf.columns]
            ax[1].bar([k.split("_")[0] for k in VIBE_KEYS], vibe_vals, color="#00A699")
            ax[1].set_title(f"Why #{1} Won ({str(winner['name'])[:15]}...)")
            ax[1].set_ylim(0, 1.1)
            top_vibe = VIBE_KEYS[np.argmax(vibe_vals)].split("_")[0]
            ax[1].set_xlabel(f"\nInsight: Strongest in {top_vibe}")

            # 3. Price Dist
            ax[2].hist(full_pdf["price"], bins=10, color='#FF5A5F', alpha=0.7, edgecolor='white', rwidth=0.85)
            ax[2].set_title("Price Distribution")
            ax[2].set_xlabel(f"Price ($)\nInsight: Avg top is ${int(avg_top_p)}")
            
            # 4. Score Dist
            ax[3].hist(full_pdf["Personalized_Score"], bins=10, color='#FC642D', alpha=0.7, edgecolor='white', rwidth=0.85)
            ax[3].set_title("Score Distribution")
            ax[3].set_xlabel(f"Score\nInsight: Tighter = Competitive")
            
            print("\n📊 Visualization Dashboard (Scroll right ➡️)")
            display_scrollable_plot(fig)

run_btn = widgets.Button(description="Find My Listings 🚀", button_style='success', layout=LAYOUT)
run_btn.on_click(on_run_click)

# ----------------------------
# 6) Surprise Me Logic
# ----------------------------
surprise_btn = widgets.Button(description="🎲 Surprise Me!", button_style='info', layout=LAYOUT)

def on_surprise_click(b):
    # Randomize Vibe Sliders only
    for k in VIBE_KEYS:
        v_sliders[k].value = random.randint(0, 10)
    on_run_click(b)

surprise_btn.on_click(on_surprise_click)


# To use the UI, All you need to do is press 'run all' and enjoy :)

In [0]:
# ----------------------------
# 7) Final UI Assembly
# ----------------------------
controls = widgets.VBox([
    widgets.HTML("<h3>1. Location & Budget</h3>"), city_dd, budget_slider,
    widgets.HTML("<h3>2. Core Preferences</h3>"), w_price, w_rating, w_amenities, w_vibe,
    widgets.HTML("<h3>3. Neighborhood Vibe</h3>"), widgets.VBox(list(v_sliders.values())),
    widgets.HTML("<h3>4. Settings</h3>"), K_slider, 
    widgets.HTML("<br>"), 
    widgets.HBox([run_btn, surprise_btn])
])

ui = widgets.HBox([
    widgets.VBox([controls], layout=widgets.Layout(width="65%", padding="0 20px 0 0")),
    widgets.VBox([explanation_panel], layout=widgets.Layout(width="35%"))
])

display(widgets.VBox([
    widgets.HTML("<h2>🏠 Personalized Airbnb VFM Engine</h2><hr>"),
    ui,
    widgets.HTML("<hr>"),
    out
]))